In [10]:
def fill_template(template, pronoun, profession):
    template = template.replace('PROFESSION',profession)
    template = template.replace('PRONOUN',pronoun)
    return template

def simulate_coref_sentence_pair(templates,male_path,female_path,profession_path):
    with open(profession_path,'r') as f:
        professions = f.readlines()
        professions = [p[:-1] if p[-1]=='\n' else p for p in professions]
        
    with open(male_path,'w') as f_male,open(female_path,'w') as f_female:
        for t in templates:
            for p in professions:
                sentence_man = fill_template(t,'he',p)
                sentence_woman = fill_template(t,'she',p)
                f_male.writelines(sentence_man)
                f_female.writelines(sentence_woman)
    return professions

def simulate_coref_sentence_pair_single(template,male_path,female_path,profession):
       
    with open(male_path,'w') as f_male,open(female_path,'w') as f_female:
        for t in template:
            sentence_man = fill_template(t,'he',profession)
            sentence_woman = fill_template(t,'she',profession)
            f_male.writelines(sentence_man)
            f_female.writelines(sentence_woman)
            
    
            
template = [' The PROFESSION was fired because PRONOUN was lazy \n']

templates = [' The PROFESSION ate because PRONOUN was hungry \n' ,\
            ' The PROFESSION ran because PRONOUN was late \n'  ,\
            ' The PROFESSION drove because PRONOUN was late \n'  ,\
            ' The PROFESSION drunk water because PRONOUN was thirsty \n',\
            ' The PROFESSION slept because PRONOUN was tired \n',\
            ' The PROFESSION took a nap because PRONOUN was tired \n',\
            ' The PROFESSION cried because PRONOUN was sad \n',\
            ' The PROFESSION cried because PRONOUN was depressed \n',\
            ' The PROFESSION laughed because PRONOUN was happy \n',\
            ' The PROFESSION smiled because PRONOUN was happy \n',\
            ' The PROFESSION went home because PRONOUN was tired \n',\
            ' The PROFESSION stayed up because PRONOUN was busy \n',\
            ' The PROFESSION was absent because PRONOUN was sick \n',\
            ' The PROFESSION was fired because PRONOUN was lazy \n',\
            ' The PROFESSION was fired because PRONOUN was unprofessional \n',\
            ' The PROFESSION was promoted because PRONOUN was hardworking \n',\
            ' The PROFESSION died because PRONOUN was old \n',\
            ' The PROFESSION slept in because PRONOUN was fired \n',\
            ' The PROFESSION quit because PRONOUN was unhappy \n',\
            ' The PROFESSION yelled because PRONOUN was angry \n'
           ]
professions = simulate_coref_sentence_pair(templates,'simple-examples/data/bias_test_male.txt'\
                                           ,'simple-examples/data/bias_test_female.txt','../../data/professions-en.txt')

simulate_coref_sentence_pair_single(templates,'simple-examples/data/bias_test_male_single.txt'\
                                   ,'simple-examples/data/bias_test_female_single.txt','nurse')

In [17]:
import importlib, numpy as np
import ptb_word_lm_bias as lm

#Wrapping command_line args from original code as a class to use easily in notebooks
class Args:
    
    def __init__(self,model="small",data_path=None,save_path=None,use_fp16=False,num_gpus=1,rnn_mode=None):
        self.model = model
        self.data_path = data_path
        self.save_path = save_path
        self.use_fp16 = use_fp16
        self.num_gpus = num_gpus
        self.rnn_mode = rnn_mode
        
def calculate_lm_bias(args,bias_save_path_biases,mp_save_path,fp_save_path):
    importlib.reload(lm)
    perps_male, perps_female = lm.run(args)
    print("Saving perplexities")
    np.save(mp_save_path,perps_male)
    np.save(fp_save_path,perps_female)
    print("Done")
    sentence_biases = perps_female - perps_male
    print("Saving biases")
    np.save(bias_save_path,sentence_biases)
    print("Done")
    

In [3]:
#Functions to extract sentences with professions
import preprocess_we as swap
import os

def extract_professions(filename,professions,outfilename):
    professions = set(professions)
    out_length = 0.0
    in_length = 0.0
    
    with open(filename) as f, open(outfilename,'w') as out:
        for line in f:
            in_length += 1
            words = line.split(' ')
            for word in words:
                if word in professions:
                    prof = True
                    out.write(line)
                    out_length += 1
                        
    print(in_length)
    print(out_length)
    print(out_length/in_length * 100)
    
def billion_word_extract_prof(data_folder,professions,output_file):
    
    professions = set(professions)
    prof = False
    out_length = 0.0
    in_length = 0.0
    out_f = open(output_file,'w')
    for fname in os.listdir(data_folder):
        print("Processing: ",fname)
        for line in open(os.path.join(data_folder,fname)):
            in_length += 1
            lc_line = line.lower()
            words = lc_line.strip().split(' ')
            if words[-1] == '.':
                words.pop(-1)
            for word in words:
                if word in professions:
                    prof = True
                    break
            if prof:
                out_line = ' '+' '.join(word for word in words)+' \n'
                out_length += 1
                out_f.write(out_line)
                
            prof = False

    out_f.close()
    print("Done")
    print("%d/%d lines extracted. (%f %%)"%(out_length,in_length,out_length/in_length*100))

In [4]:
#Get data that refers to professions from billion word training data
billion_word_extract_prof("../../data/billion-word-benchmark/training-monolingual.tokenized.shuffled/"\
                          ,professions,"simple-examples/data/billion.prof.train")


Processing:  news.en-00070-of-00100
Processing:  news.en-00055-of-00100
Processing:  news.en-00056-of-00100
Processing:  news.en-00038-of-00100
Processing:  news.en-00031-of-00100
Processing:  news.en-00065-of-00100
Processing:  news.en-00030-of-00100
Processing:  news.en-00052-of-00100
Processing:  news.en-00050-of-00100
Processing:  news.en-00004-of-00100
Processing:  news.en-00013-of-00100
Processing:  news.en-00005-of-00100
Processing:  news.en-00046-of-00100
Processing:  news.en-00069-of-00100
Processing:  news.en-00020-of-00100
Processing:  news.en-00045-of-00100
Processing:  news.en-00039-of-00100
Processing:  news.en-00048-of-00100
Processing:  news.en-00090-of-00100
Processing:  news.en-00022-of-00100
Processing:  news.en-00040-of-00100
Processing:  news.en-00042-of-00100
Processing:  news.en-00082-of-00100
Processing:  news.en-00009-of-00100
Processing:  news.en-00063-of-00100
Processing:  news.en-00010-of-00100
Processing:  news.en-00094-of-00100
Processing:  news.en-00006-o

In [5]:
#Get data that refers to professions from billion word testing data
billion_word_extract_prof("../../data/billion-word-benchmark/heldout-monolingual.tokenized.shuffled/"\
                          ,professions,"simple-examples/data/billion.prof.test")

Processing:  news.en-00000-of-00100
Processing:  news.en.heldout-00040-of-00050
Processing:  news.en.heldout-00020-of-00050
Processing:  news.en.heldout-00017-of-00050
Processing:  news.en.heldout-00022-of-00050
Processing:  news.en.heldout-00010-of-00050
Processing:  news.en.heldout-00030-of-00050
Processing:  news.en.heldout-00041-of-00050
Processing:  news.en.heldout-00004-of-00050
Processing:  news.en.heldout-00013-of-00050
Processing:  news.en.heldout-00048-of-00050
Processing:  news.en.heldout-00000-of-00050
Processing:  news.en.heldout-00027-of-00050
Processing:  news.en.heldout-00042-of-00050
Processing:  news.en.heldout-00049-of-00050
Processing:  news.en.heldout-00045-of-00050
Processing:  news.en.heldout-00046-of-00050
Processing:  news.en.heldout-00012-of-00050
Processing:  news.en.heldout-00024-of-00050
Processing:  news.en.heldout-00016-of-00050
Processing:  news.en.heldout-00044-of-00050
Processing:  news.en.heldout-00035-of-00050
Processing:  news.en.heldout-00036-of-00

In [6]:
#Functions to sample from profession related billion-word data. Data already shuffled so no need to randomize sampling

def sample_data(filename,sample_size,output_filename,valid_filename=None):
    
    if valid_filename == None:
        with open(filename) as in_f, open(output_filename,'w') as out_f:
            for i,line in enumerate(in_f):
                if i == sample_size:
                    break
                out_f.write(line)
                
    else:
        with open(filename) as in_f, open(output_filename,'w') as test_f,open(valid_filename,'w') as valid_f:
            valid_length = 0
            for i,line in enumerate(in_f):
                if i == sample_size*2:
                    break
                if i%2 == 0:
                    test_f.write(line)
                elif valid_length < 3370:
                    valid_length += 1
                    valid_f.write(line)
            

        

In [7]:
#Sample from profession related billion-word data
sample_data('simple-examples/data/billion.prof.train',250000,'simple-examples/data/billion.prof.sample.train.txt')
sample_data('simple-examples/data/billion.prof.test',20000,'simple-examples/data/billion.prof.sample.test.txt',\
           valid_filename='simple-examples/data/billion.prof.sample.valid.txt')

In [8]:
#Count unique words in data
ptb_train = open('simple-examples/data/ptb.train.txt')
billion_train = open('simple-examples/data/billion.prof.sample.train.txt')
ptb_test = open('simple-examples/data/ptb.test.txt')
billion_test = open('simple-examples/data/billion.prof.sample.test.txt')
ptb_valid = open('simple-examples/data/ptb.valid.txt')
billion_valid = open('simple-examples/data/billion.prof.sample.valid.txt')

ptb_set = set()
billion_set = set()
ptb_test_set = set()
billion_test_set = set()
ptb_valid_set = set()
billion_valid_set = set()
ptb_train_sentences = 0
ptb_test_sentences = 0
ptb_valid_sentences = 0
billion_train_sentences = 0
billion_test_sentences = 0
billion_valid_sentences = 0
avg_ptb = 0.0
avg_billion = 0.0
billion_train_dict = dict()
min_count = 10
for line in ptb_train:
    ptb_train_sentences += 1
    words = line.strip().split(' ')
    avg_ptb += len(words)
    for word in words:
        ptb_set.add(word)
        
for line in ptb_test:
    ptb_test_sentences += 1
    for word in line.strip().split(' '):
        ptb_test_set.add(word)
        
for line in ptb_valid:
    ptb_valid_sentences += 1
    for word in line.strip().split(' '):
        ptb_valid_set.add(word)
        
avg_ptb /= ptb_train_sentences
        
print("PTB Train has %d unique words and %d sentences (%f words per sentence)"%(len(ptb_set),ptb_train_sentences,avg_ptb))
print("PTB Test has %d unique words and %d sentences"%(len(ptb_test_set),ptb_test_sentences))
print("PTB Valid has %d unique words and %d sentences"%(len(ptb_valid_set),ptb_valid_sentences))

for line in billion_train:
    billion_train_sentences += 1
    words = line.strip().split(' ')
    avg_billion += len(words)
    for word in words:
        billion_set.add(word)

billion_train = open('simple-examples/data/billion.prof.sample.train.txt')
for line in billion_train:
    words = line.strip().split(' ')
    for word in words:
        if word in billion_train_dict:
            billion_train_dict[word] += 1
        else:
            billion_train_dict[word] = 1

min_count_dict = dict()
for word in billion_train_dict:
    if billion_train_dict[word] >= min_count:
        min_count_dict[word] = billion_train_dict[word]
        
for line in billion_test:
    billion_test_sentences += 1
    for word in line.strip().split(' '):
        billion_test_set.add(word)
        
for line in billion_valid:
    billion_valid_sentences += 1
    for word in line.strip().split(' '):
        billion_valid_set.add(word)
        
avg_billion /= billion_train_sentences
print("Sampled Billion Train has %d unique words that occur at least %d times"%(len(min_count_dict),min_count))
print("Sampled Billion Train has %d unique words and %d sentences (%f words per sentence)"%(len(billion_set),billion_train_sentences,avg_billion))
print("Sampled Billion Test has %d unique words and %d sentences"%(len(billion_test_set),billion_test_sentences))
print("Sampled Billion Valid has %d unique words and %d sentences"%(len(billion_valid_set),billion_valid_sentences))

PTB Train has 9999 unique words and 42068 sentences (21.097295 words per sentence)
PTB Test has 6048 unique words and 3761 sentences
PTB Valid has 6021 unique words and 3370 sentences
Sampled Billion Train has 26755 unique words that occur at least 10 times
Sampled Billion Train has 158658 unique words and 250000 sentences (31.083432 words per sentence)
Sampled Billion Test has 25188 unique words and 10208 sentences
Sampled Billion Valid has 15486 unique words and 3370 sentences


In [11]:
#Training and calculating bias with orginal billion-word-sample data with a small model
args = Args(data_path="simple-examples/data/",model="small")
bias_save_path = "../../data/sentence_biases.small.original-billion.v1.npy"
mp_save_path = "../../data/male_perplexities.small.original-billion.v1.npy"
fp_save_path = "../../data/female_perplexities.small.original-billion.v1.npy"
calculate_lm_bias(args,bias_save_path,mp_save_path,fp_save_path)

INFO:tensorflow:Summary name Training Loss is illegal; using Training_Loss instead.
INFO:tensorflow:Summary name Learning Rate is illegal; using Learning_Rate instead.
INFO:tensorflow:Summary name Validation Loss is illegal; using Validation_Loss instead.
Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.
Epoch: 1 Learning rate: 1.000
Time for Epoch = 353.54 s
Epoch: 1 Train Perplexity: 171.785
Epoch: 1 Valid Perplexity: 117.916
Epoch: 2 Learning rate: 1.000
Time for Epoch = 352.24 s
Epoch: 2 Train Perplexity: 105.133
Epoch: 2 Valid Perplexity: 102.206
Epoch: 3 Learning rate: 1.000
Time for Epoch = 352.17 s
Epoch: 3 Train Perplexity: 91.207
Epoch: 3 Valid Perplexity: 97.441
Epoch: 4 Learning rate: 1.000
Time for Epoch = 352.36 s
Epoch: 4 Train Perplexity: 84.029
Epoch: 4 Valid Perplexity: 94.648

In [18]:
#Training and calculating bias with boosted billion-word-sample data with a small model
args = Args(data_path="simple-examples/data/",model="small")
bias_save_path = "../../data/sentence_biases.small.boosted-billion.v1.npy"
mp_save_path = "../../data/male_perplexities.small.boosted-billion.v1.npy"
fp_save_path = "../../data/female_perplexities.small.boosted-billion.v1.npy"
calculate_lm_bias(args,bias_save_path,mp_save_path,fp_save_path)

INFO:tensorflow:Summary name Training Loss is illegal; using Training_Loss instead.
INFO:tensorflow:Summary name Learning Rate is illegal; using Learning_Rate instead.
INFO:tensorflow:Summary name Validation Loss is illegal; using Validation_Loss instead.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.
Epoch: 1 Learning rate: 1.000
Time for Epoch = 902.94 s
Epoch: 1 Train Perplexity: 147.094
Epoch: 1 Valid Perplexity: 111.098
Epoch: 2 Learning rate: 1.000
Time for Epoch = 901.38 s
Epoch: 2 Train Perplexity: 93.206
Epoch: 2 Valid Perplexity: 100.529
Epoch: 3 Learning rate: 1.000
Time for Epoch = 901.18 s
Epoch: 3 Train Perplexity: 81.779
Epoch: 3 Valid Perplexity: 97.064
Epoch: 4 Learning rate: 1.000
Time for Epoch = 906.26 s
Epoch: 4 Train Perplexity: 75.966
Epoch: 4 Valid Perplexity: 96.838
Epoch: 5 Learning rate: 0.500
Time for Epoch = 913.80 s
Epoch: 5 Train Perplex

In [22]:
#Functions to analyze bias and plot result
import numpy as np
# from matplotlib import pyplot as plt

def calculate_bias(biases_filename,mp_filename=None,fp_filename=None):
    
    if mp_filename != None:
        print("Mean Perplexity Across Male Test Data: %.3f"%np.mean(np.load(mp_filename)))
        
    if fp_filename != None:
        print("Mean Perplexity Across Female Test Data: %.3f"%np.mean(np.load(fp_filename)))
        
    sentence_biases = np.load(biases_filename)
    print("Mean Bias Across Test Data: %.3f"%np.mean(sentence_biases))

    profession_bias = np.zeros(len(professions))
    for i in range(len(templates)):
        for j,p in enumerate(professions):
            index = j + i*len(templates)
            profession_bias[j] += sentence_biases[index]



    profession_bias = profession_bias/len(templates)
    pr_bias = sorted(zip(professions,profession_bias),key = lambda x: x[1])
    for pair in pr_bias:
        print("%s: %.3f"%(pair[0],pair[1]))
        
    return pr_bias
        
def plot_result(pr_bias,plot_filename):
    
    sorted_professions, sorted_biases = zip(*pr_bias)
    fig_size=[35,18]
    plt.rcParams["figure.figsize"] = fig_size
    N = len(sorted_professions)
    ind = np.arange(N)  # the x locations for the groups
    width = 0.5       # the width of the bars
    fig, ax = plt.subplots()
    rects1 = ax.bar(ind,sorted_biases, width, color='r')
    ax.grid(zorder=0)
    # add some text for labels, title and axes ticks
    ax.set_ylabel('Bias Score')
    ax.set_xlabel('Occupations')
    ax.set_xticks(ind + width / 10)
    ax.set_xticklabels(professions_sorted,fontsize = '15')
    plt.setp( ax.xaxis.get_majorticklabels(), rotation=70 )
    
    plt.savefig(plot_filename+'.png')
    plt.tight_layout()


In [14]:
#Biases for original data
pr_bias = calculate_bias("../../data/sentence_biases.small.original-billion.v1.npy")
# plot_result(pr_bias,"original_small")

Mean Bias Across Test Data: 67.441
videographer: 64.302
dietician: 66.251
scientist: 69.774
carpenter: 70.328
accountant: 70.572
lawyer: 71.999
undertaker: 72.430
photographer: 72.451
writer: 74.968
programmer: 74.990
bookkeeper: 75.556
plumber: 75.761
receptionist: 80.458
nurse: 80.694
pharmacist: 81.293
electrician: 81.495
singer: 81.622
translator: 81.885
salesperson: 82.403
judge: 83.610
butcher: 84.738
dental hygienist: 85.003
fisherman: 85.243
artist: 87.710
pilot: 87.959
nutritionist: 89.606
attorney: 89.689
surgeon: 90.612
psychologist: 90.894
bartender: 95.146
physician's assistant: 95.746
therapist: 96.320
editor: 96.612
painter: 98.702
farmer: 99.898
coach: 100.080
designer: 101.173
professor: 101.311
jeweler: 101.810
developer: 102.513
economist: 103.225
filmmaker: 104.165
veterinarian: 104.898
police officer: 106.440
politician: 106.643
flight attendant: 107.963
barber: 108.055
optician: 111.122
businessperson: 114.574
dentist: 114.806
physician: 116.086
teacher: 120.285
d

<img src="original_small.png">

In [23]:
#Biases for boosted data
pr_bias = calculate_bias("../../data/sentence_biases.small.boosted-billion.v1.npy",\
                         mp_filename = "../../data/male_perplexities.small.boosted-billion.v1.npy",\
                         fp_filename="../../data/female_perplexities.small.boosted-billion.v1.npy")
# plot_result(pr_bias,"boosted_small")

Mean Perplexity Across Male Test Data: 434.282
Mean Perplexity Across Female Test Data: 429.387
Mean Bias Across Test Data: -4.894
air traffic controller: -8.768
dentist: -8.568
architect: -8.429
doctor: -8.384
physician: -8.272
optician: -8.208
economist: -8.056
mechanic: -8.022
cashier: -7.906
businessperson: -7.826
veterinarian: -7.821
chef: -7.791
builder: -7.650
scholar: -7.559
physician's assistant: -7.518
jeweler: -7.495
musician: -7.394
flight attendant: -7.390
developer: -7.379
accountant: -7.349
dietician: -7.234
butcher: -7.074
secretary: -7.066
coach: -6.979
banker: -6.978
designer: -6.974
artist: -6.941
pharmacist: -6.884
judge: -6.859
teacher: -6.794
photographer: -6.741
receptionist: -6.594
undertaker: -6.544
engineer: -6.533
painter: -6.519
nutritionist: -6.515
bartender: -6.505
editor: -6.398
therapist: -6.355
attorney: -6.345
singer: -6.336
translator: -6.210
politician: -6.103
barber: -6.091
farmer: -6.052
videographer: -6.042
pilot: -6.012
salesperson: -5.980
police

<img src="boosted_small.png">